In [59]:
import os
import pandas as pd

# show all columns
pd.options.display.max_columns = None

In [60]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
  
outputs = '.\\Outputs'

In [61]:
def run_bases(_batResult, _runner1, _runner2, _runner3, _outs):
    _runs = 0 # track runs
    _new_outs = _outs

    # if home run
    if _batResult == 4:
        
        # score a run for the hitter
        _runs = _runs + 1 

        # if runner is on base, runner scores and base count is reset
        if (_runner1 > 0):
                _runs = _runs + 1
                _runner1 = 0
        
        if (_runner2 > 0):
                _runs = _runs + 1
                _runner2 = 0

        if (_runner3 > 0):
                _runs = _runs + 1
                _runner3 = 0

        # return runners, runs, outs
        return [_runner1, _runner2, _runner3, _runs, _new_outs]
    
    # if single, double, or triple
    if(_batResult > 0 and _batResult < 4):
            
        # if no runners on bases
        if ((_runner1 == 0) and (_runner2 == 0) and _runner3 == 0):
            _runner1 = _runner1 + _batResult
        
        # if runner(s) on base
        else:

            # runner1
            if (_runner1 > 0):
                _runner1 = _runner1 + _batResult
                    
                # if runner has exceed bases, runner scores and base count is reset
                if(_runner1 > 3):
                    _runner1 = 0
                    _runs = _runs + 1  
            
            # runner2
            if (_runner2 > 0):
                _runner2 = _runner2 + _batResult
                    
                if(_runner2 > 3):
                    _runner2 = 0
                    _runs = _runs + 1 

            # runner23
            if (_runner3 > 0):
                _runner3 = _runner3 + _batResult
                    
                if(_runner3 > 3):
                    _runner3 = 0
                    _runs = _runs + 1   
            
            #==========================
            # find the new runner slot
            #==========================
            
            if (_runner1 == 0):
                    _runner1 = _runner1 + _batResult
    
            elif (_runner2 == 0):
                    _runner2 = _runner2 + _batResult

            elif (_runner3 == 0):
                    _runner3 = _runner3 + _batResult
                  
        # return runners and runs
        return [_runner1, _runner2, _runner3, _runs, _new_outs]
    
    # if out 
    else:
        if(_outs >= 2):
            _new_outs = 0
            _runner1 = 0
            _runner2 = 0
            _runner3 = 0
        else:
            _new_outs = _outs + 1 
        
        # return runners and runs
        return [_runner1, _runner2, _runner3, _runs, _new_outs]

In [62]:
#################
# test one team
#################

# bats_all = pd.read_csv('bats1.csv')
# team_id = 4
# bats_team = bats_all[bats_all['team_id']==team_id]
# bats_team.reset_index()

# team_dict = {'team_id':team_id, 'runner1':0, 'runner2':0, 'runner3':0, 'outs':0, 'runs':0}

# for index, row in bats_team.iterrows():
#     result = run_bases(row['bat_result'], team_dict['runner1'], team_dict['runner2'],team_dict['runner3'], team_dict['outs']) 
    
#     team_dict['runner1'] = result[0]
#     team_dict['runner2'] = result[1]
#     team_dict['runner3'] = result[2]
#     team_dict['outs'] = result[4]
#     team_dict['runs'] = team_dict['runs'] + result[3]

# team_dict

In [63]:
# arguments
in_table = 'inputs/bats4.csv'
out_table = 'week4.csv'

In [64]:
bats_all = pd.read_csv(in_table)
teams_ids = list(set(bats_all['team_id'].to_list()))
team_results = []

for team_id in teams_ids:
    
    team = {'team_id':team_id, 'runner1':0, 'runner2':0, 'runner3':0, 'outs':0, 'runs':0}
    bats_team = bats_all[bats_all['team_id']==team_id].copy()

    for index, row in bats_team.iterrows():
        result = run_bases(row['bat_result'], team['runner1'], team['runner2'],team['runner3'], team['outs']) 
        team['runner1'] = result[0]
        team['runner2'] = result[1]
        team['runner3'] = result[2]
        team['outs'] = result[4]
        team['runs'] = team['runs'] + result[3]

    team_results.append(team)

In [66]:
scores = pd.DataFrame(team_results).sort_values('runs', ascending=False)
scores = scores.merge(pd.read_csv('inputs/teams.csv'), on='team_id', how='outer')
scores = scores.fillna(0)
scores = scores[['team_id', 'name', 'runner1', 'runner2', 'runner3', 'outs', 'runs']]

# export results
scores.to_csv(os.path.join(outputs, out_table), index=False)